<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<center><img src = "https://github.com/Jangrae/img/blob/master/silver.jpg?raw=true" width=800/></center>

# 실습 내용

- Silver 데이터로 모델링합니다.
- KNN 알고리즘으로 모델링합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

In [2]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/silver_simple.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [3]:
# 상위 몇 개 행 확인
data.head()

,사고발생여부,인구수,생활인구수_주중,생활인구수_주말,전통시장여부,지하철출입구개수,버스정류장개수,과속방지턱개수,교차로개수,횡단보도개수,주거,상업,녹지
0,1,2972.465549,569.569424,586.464251,0,0,0,0,0,0,0.999866,0.000000,0.00000
1,1,1122.540470,307.268617,328.396582,0,0,0,0,0,0,0.727125,0.000000,0.00000
2,1,0.000000,636.368613,675.918350,0,1,1,1,0,3,0.716928,0.249503,0.02829
3,1,0.000000,583.986187,598.863080,0,0,0,0,0,0,1.000035,0.000000,0.00000
4,0,1973.416627,736.053774,728.240249,0,0,1,0,0,0,0.902334,0.000000,0.00000


In [4]:
# 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   사고발생여부    2000 non-null   int64  
 1   인구수       2000 non-null   float64
 2   생활인구수_주중  2000 non-null   float64
 3   생활인구수_주말  2000 non-null   float64
 4   전통시장여부    2000 non-null   int64  
 5   지하철출입구개수  2000 non-null   int64  
 6   버스정류장개수   2000 non-null   int64  
 7   과속방지턱개수   2000 non-null   int64  
 8   교차로개수     2000 non-null   int64  
 9   횡단보도개수    2000 non-null   int64  
 10  주거        2000 non-null   float64
 11  상업        2000 non-null   float64
 12  녹지        2000 non-null   float64
dtypes: float64(6), int64(7)
memory usage: 203.3 KB


In [5]:
# 기술통계 확인
data.describe()

,사고발생여부,인구수,생활인구수_주중,생활인구수_주말,전통시장여부,지하철출입구개수,버스정류장개수,과속방지턱개수,교차로개수,횡단보도개수,주거,상업,녹지
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.500000,666.051466,407.189549,398.327882,0.084000,0.050500,0.406000,0.313500,0.265000,0.959500,0.829199,0.068018,0.050525
std,0.500125,944.927905,179.293956,186.297298,0.277457,0.411138,0.747291,0.722827,0.490811,1.323153,0.229318,0.153374,0.117042
min,0.000000,0.000000,37.452857,32.994461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,280.401103,260.086352,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.741707,0.000000,0.000000
50%,0.500000,0.000000,394.438137,368.964573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.931118,0.000000,0.000000
75%,1.000000,1399.210518,527.348901,523.103365,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.999803,0.077499,0.015522
max,1.000000,3094.897898,972.048024,1009.675570,1.000000,10.000000,4.000000,5.000000,2.000000,8.000000,1.003925,1.000531,0.873327


In [6]:
# 범줏값 개수 확인
data['사고발생여부'].value_counts()

사고발생여부
1    1000
0    1000
Name: count, dtype: int64

In [7]:
# 결측치 확인
data.isnull().sum()

사고발생여부      0
인구수         0
생활인구수_주중    0
생활인구수_주말    0
전통시장여부      0
지하철출입구개수    0
버스정류장개수     0
과속방지턱개수     0
교차로개수       0
횡단보도개수      0
주거          0
상업          0
녹지          0
dtype: int64

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [8]:
# target 확인
target = '사고발생여부'   # <- 실제 target 변수 이름으로 변경해주세요
print("Target:", target)

# 데이터 분리
x = data.drop(columns=[target])
y = data[target]

# 확인
print(x.shape, y.shape)
x.head(), y.head()

Target: 사고발생여부
(2000, 12) (2000,)


(           인구수    생활인구수_주중    생활인구수_주말  전통시장여부  지하철출입구개수  버스정류장개수  과속방지턱개수  \
 0  2972.465549  569.569424  586.464251       0         0        0        0   
 1  1122.540470  307.268617  328.396582       0         0        0        0   
 2     0.000000  636.368613  675.918350       0         1        1        1   
 3     0.000000  583.986187  598.863080       0         0        0        0   
 4  1973.416627  736.053774  728.240249       0         0        1        0   
 
    교차로개수  횡단보도개수        주거        상업       녹지  
 0      0       0  0.999866  0.000000  0.00000  
 1      0       0  0.727125  0.000000  0.00000  
 2      0       3  0.716928  0.249503  0.02829  
 3      0       0  1.000035  0.000000  0.00000  
 4      0       0  0.902334  0.000000  0.00000  ,
 0    1
 1    1
 2    1
 3    1
 4    0
 Name: 사고발생여부, dtype: int64)

**2) 학습용, 평가용 데이터 분리**

In [10]:
# 모듈 불러오기
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

**3-2) 스케일링 - 표준화(StandardScaler)**

- 표준화는 다음 공식에 의해 각 변수의 평균이 0, 표준편차가 1이 되게 합니다.
$$ \large x_z = \frac{x - x_{mean}}{x_{std}} $$
- 위 정규화 대신 표준화 과정 진행 후 성능 차이를 확인합니다.

In [11]:
# 모듈 불러오기
from sklearn.preprocessing import StandardScaler

# 정규화
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.

In [12]:
# 1단계: 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
# 2단계: 선언하기
model = KNeighborsClassifier()

In [14]:
# 3단계: 학습하기
model.fit(x_train, y_train)

KNeighborsClassifier()

In [15]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [16]:
# 5단계: 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[253  47]
 [ 71 229]]
              precision    recall  f1-score   support

           0       0.78      0.84      0.81       300
           1       0.83      0.76      0.80       300

    accuracy                           0.80       600
   macro avg       0.81      0.80      0.80       600
weighted avg       0.81      0.80      0.80       600

